In [1]:
conda activate kdm

In [2]:
bcftools view -h input/freebayes_at9852_ath_filt-q100-ac10-dp100-200k.vcf.gz | \
    grep '^#CHROM' | tr '\t' '\n' | tail +10 |\
    grep "^OAKG" > input/only_1k1g_samples.txt

In [3]:
mkdir -p output

In [13]:
blsl gffcsqify  -o output/at9852_hodgepodge-representative.gff3 ../final/annotation/latest/at9852_hodgepodgemerged.representatives.gff3 

In [18]:
rm -f  output/at9852_hodgepodge-representative.phasefix.gff3 
agat_sp_fix_cds_phases.pl -g output/at9852_hodgepodge-representative.gff3 -f ../final_scaffolds/at9852_linewrapped.fasta -o  output/at9852_hodgepodge-representative.phasefix.gff3 

Using standard /tmp/global2/kmurray/conda/envs/kdm/lib/perl5/site_perl/auto/share/dist/AGAT/agat_config.yaml file

 ------------------------------------------------------------------------------
|   Another GFF Analysis Toolkit (AGAT) - Version: v1.2.0                      |
|   https://github.com/NBISweden/AGAT                                          |
|   National Bioinformatics Infrastructure Sweden (NBIS) - www.nbis.se         |
 ------------------------------------------------------------------------------
                                        
                                       
                          ------ Start parsing ------                           
-------------------------- parse options and metadata --------------------------
=> Accessing the feature_levels YAML file
Using standard /tmp/global2/kmurray/conda/envs/kdm/lib/perl5/site_perl/auto/share/dist/AGAT/feature_levels.yaml file
=> Attribute used to group features when no Parent/ID relationship exists (i.e c

In [4]:
awk 'BEGIN{FS=OFS="\t"}$3 == "gene"{print $1, $4-1, $5, gensub(/ID=([^;]+).*/, "\\1", "g", $9)}' ../final/annotation/latest/at9852_hodgepodgemerged.representatives.gff3 \
 | bedtools slop -b 100 -i - -g ../final_scaffolds/at9852.fasta.fai \
 >output/all_genes_9852_slop_100.bed

In [6]:
blsl vcfparallel \
    -o output/freebayes_at9852_ath_filt-q100-ac10-dp100-200k-only1k1g-csq_allgenes_csqtab.tsv \
    -R output/all_genes_9852_slop_100.bed \
    -T /ebio/scratch/kmurray/tmp/bcffilt/ \
    -f "-S input/only_1k1g_samples.txt" \
    --merge-with-cat \
    -c "bcftools norm -N -m -any  /dev/stdin | bcftools csq -Ob0 --phase s --gff output/at9852_hodgepodge-representative.phasefix.gff3 --fasta ../final_scaffolds/at9852.fasta /dev/stdin | bcftools query -f '%CHROM\t%POS\t%POS\t%ALT\t%QUAL\t%AF\t%BCSQ\n'" \
    -t 128  \
    input/freebayes_at9852_ath_filt-q100-ac10-dp100-200k.vcf.gz

Using temporary file prefix: /ebio/scratch/kmurray/tmp/bcffilt/9094983c-c39c-4762-96af-af0f7835dbac. This WILL NOT be automatically cleaned!!
merge with cat: 100%|█████████████████| 26770/26770 [00:13<00:00, 1957.70file/s]


In [7]:
ls -lahF output/

total 785M
drwxr-xr-x 2 kmurray ebio           4.0K Nov 18 15:06 ./
drwxr-x--- 6 kmurray p-diffLines_20  327 Nov 18 15:05 ../
-rw-r--r-- 1 kmurray ebio           1.2M Nov 18 14:46 all_genes_9852_slop_100.bed
lrwxrwxrwx 1 kmurray ebio            208 Nov  7 15:12 at9852_hodgepodge-representative.gff3 -> ../../.git/annex/objects/7X/q4/SHA256E-s70159464--dd7d51737a1c87b6bb694bb46abe5148ea37e40f78513745fde1a48d4a6c74a6.gff3/SHA256E-s70159464--dd7d51737a1c87b6bb694bb46abe5148ea37e40f78513745fde1a48d4a6c74a6.gff3
lrwxrwxrwx 1 kmurray ebio            208 Nov  7 15:19 at9852_hodgepodge-representative.phasefix.gff3 -> ../../.git/annex/objects/22/Qx/SHA256E-s71053444--18bb71dcb0b2ef58fd1e56b8a0a3b9bb725450ba9ee9fa22f247fe15dd959cec.gff3/SHA256E-s71053444--18bb71dcb0b2ef58fd1e56b8a0a3b9bb725450ba9ee9fa22f247fe15dd959cec.gff3
-rw-r--r-- 1 kmurray ebio           602M Nov 18 15:06 freebayes_at9852_ath_filt-q100-ac10-dp100-200k-only1k1g-csq_allgenes_csqtab.tsv
lrwxrwxrwx 1 kmurray ebio            206 

In [69]:
bedtools intersect \
    -a ./output/validated_nlrs_only_9852_slop100bp.bed \
    -b output/freebayes_at9852_ath_filt-q100-ac10-dp100-200k-only1k1g-csq_onlyNLRs_csqtab.tsv \
    -loj \
    | awk -v 'OFS=\t' '{print $5, $6, $8, $9, $10, $11, $4}' \
    > output/freebayes_at9852_ath_filt-q100-ac10-dp100-200k-only1k1g-csq_onlyNLRs_csqtab_geneids.tsv

In [70]:
head output/freebayes_at9852_ath_filt-q100-ac10-dp100-200k-only1k1g-csq_onlyNLRs_csqtab_geneids.tsv
wc -l output/freebayes_at9852_ath_filt-q100-ac10-dp100-200k-only1k1g-csq_onlyNLRs_csqtab_geneids.tsv

at9852_1_chr1	3151589	C	5386.12	0.0155675	.	at9852_G009400
at9852_1_chr1	3151591	C	5386.12	0.00251088	.	at9852_G009400
at9852_1_chr1	3151613	A	489.192	0.00181938	.	at9852_G009400
at9852_1_chr1	3151637	A	16241.1	0.0129741	.	at9852_G009400
at9852_1_chr1	3151662	C	656307	0.225697	.	at9852_G009400
at9852_1_chr1	3151662	*	656307	.	.	at9852_G009400
at9852_1_chr1	3151671	G	656307	0.274813	.	at9852_G009400
at9852_1_chr1	3151671	*	656307	.	.	at9852_G009400
at9852_1_chr1	3151674	A	656307	0.229096	.	at9852_G009400
at9852_1_chr1	3151674	*	656307	.	.	at9852_G009400
101617 output/freebayes_at9852_ath_filt-q100-ac10-dp100-200k-only1k1g-csq_onlyNLRs_csqtab_geneids.tsv
